In [ ]:
import requests
import json
import sys
import os

In [ ]:
DEBUG = True
AOS_HOST = '10.60.3.20'
HCL_ENTRIES_FOLDER = '/var/hydra/aos/hcl'
API_AUTH_ENDPOINT = '/api/aaa/login'
API_HCL_ENDPOINT = '/api/hcl'

In [ ]:
data = { 'username' : 'admin', 'password' : raw_input() }
r = requests.post('https://{0}{1}'.format(AOS_HOST,API_AUTH_ENDPOINT), data=json.dumps(data), verify=False)
token = json.loads(r.text)['token']
id = json.loads(r.text)['id']

In [ ]:
headers = {'AUTHTOKEN': token, 'id': id}
r = requests.get('https://{0}{1}'.format(AOS_HOST,API_HCL_ENDPOINT), headers=headers, verify=False)
json_data = json.loads(r.text)
aos_hcls_dict = {}
for item in json_data['items']:
    item.pop(u'last_modified_at')
    item.pop(u'created_at')
    aos_hcls_dict[item[u'id']] = item

In [ ]:
def add_hcl_aos(u_token,u_id,hcl):
    headers = {'AUTHTOKEN': u_token, 'id': u_id, 'Content-type': 'application/json', 'Accept': 'application/json'}
    r = requests.post('https://{0}{1}'.format(AOS_HOST,API_HCL_ENDPOINT), headers=headers, json=hcl, verify=False)
    return r

In [ ]:
def update_hcl_aos(u_token,u_id,hcl):
    headers = {'AUTHTOKEN': u_token, 'id': u_id, 'Content-type': 'application/json', 'Accept': 'application/json'}
    r = requests.put('https://{0}{1}/{2}'.format(AOS_HOST,API_HCL_ENDPOINT,hcl[u'id']), headers=headers, json=hcl, verify=False)
    return r

In [ ]:
local_hcl_json_data = []
added_counter = 0
updated_counter = 0
for fn in os.listdir(HCL_ENTRIES_FOLDER):
    f=open(os.path.join(HCL_ENTRIES_FOLDER,fn),'r') 
    json_data=f.read()
    hcl = json.loads(json_data)
    local_hcl_json_data.append(hcl)
    if hcl['id'] not in aos_hcls_dict.keys():
        print 'HCL {} needs to be added'.format(hcl['id'])
        r = add_hcl_aos(token,id,hcl)
        if not r.status_code == requests.codes.ok:
            raise Exception('HCL {} could not be added: response code = {}'.format(hcl['id'],r.status_code))
        else:
            added_counter +=1
    elif not hcl == aos_hcls_dict[hcl['id']]:
        print 'HCL {} needs to be updated'.format(hcl['id'])
        r = update_hcl_aos(token,id,hcl)
        print r.status_code
        if not r.status_code == requests.codes.ok:
            raise Exception('HCL {} could not be updated: response code = {}'.format(hcl['id'],r.status_code))
        else:
            updated_counter +=1

print "A Total of {} HCLs were added".format(added_counter)
print "A Total of {} HCLs were updated".format(updated_counter)